In [2]:
from qpms import Particle, CTMatrix, lorentz_drude, EpsMuGenerator, TMatrixGenerator, BaseSpec, FinitePointGroup, ScatteringSystem, TMatrixInterpolator, EpsMu, dbgmsg_enable, dbgmsg_disable, dbgmsg_active, BesselType,eV, hbar, c
from qpms.symmetries import point_group_info
import math
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
import numpy as np
eh = eV/hbar
np.random.seed(666)
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Rectangle, Circle
from mpl_toolkits.axes_grid1 import AxesGrid

def cleanarray(a, atol=1e-10, copy=True):
    a = np.array(a, copy=copy)
    sieve = abs(a.real) < atol
    a[sieve] = 1j * a[sieve].imag
    sieve = abs(a.imag) < atol
    a[sieve] = a[sieve].real
    return a

In [3]:
dbgmsg_enable(2)
#part_radius = 80e-9

<DebugFlags.THREADS|MISC: 3>

In [4]:
#modefile = '/u/46/necadam1/unix/repo/qpms/misc/modes_basis300nm_0nm__0nm_400nm_n1_b-2_k(0_0)um-1_cn150.npz' # PARAM!
#modefile='/l/necadam1/test_mode_fieldplots/cyl_r50nm_h50nm_p560nmx560nm_mAu_n1.52_b-2_k(0_0)um-1_L3_cn200.npz'
modefile='/repo/tmp/cyl_r50nm_h50nm_p560nmx560nm_mAu_n1.52_b-2_k(0_0)um-1_L3_cn200.npz'
#modefile = '/l/necadam1/sqlat-Tommi/sph_r80nm_p580nm_m5.0_n1.52_b-3_k(0_0)um-1_L3_cn3100.npz'
data = np.load(modefile, allow_pickle=True)
meta = data['meta'][()]
list(data.keys())

['meta',
 'empty_freqs',
 'ss_positions',
 'ss_fullvec_poffsets',
 'ss_fullvec_psizes',
 'ss_bspecs_flat',
 'ss_lattice_basis',
 'ss_reciprocal_basis',
 'eigval',
 'eigval_inside_metric',
 'eigvec',
 'residuals',
 'eigval_err',
 'ranktest_SV',
 'iri',
 'blochvector',
 'inside_contour',
 'refractive_index_internal']

In [4]:
# initialise the medium and dummy t-matrix generators, so we can "reconstruct" the ScatSystem instance (only background medium is relevant here)
medium = EpsMuGenerator(meta['background'])
emg_dummy = EpsMuGenerator(10)
tmg_dummy = TMatrixGenerator.sphere(medium, emg_dummy, 50e-9)
try: default_omega = data['eigval'][0]
except IndexError as ie:
    raise ValueError("Data file contains no eigenvalues") from ie
    

In [5]:
# create (periodic) ss, ssw; init wavevector to use with sswk constructor
particles = []
offsets = data['ss_fullvec_poffsets']
nelems = data['ss_fullvec_psizes']
for pi, pos in enumerate(data['ss_positions']):
    bspec = BaseSpec(data['ss_bspecs_flat'][offsets[pi]:offsets[pi]+nelems[pi]])
    particles.append(Particle(pos, tmg_dummy, bspec))
ss, ssw = ScatteringSystem.create(particles, medium, default_omega, latticebasis = data['ss_lattice_basis'])
wavevector = np.array([0., 0., 0.]) # it needs to be 3D for sswk "constructor", 
for i in range(len(meta['k'])):
    wavevector[i] = meta['k'][i]
    
# determine the "unit rectangle" size for plotting
lb = ss.lattice_basis
xmax = max(abs(lb[0,0]), abs(lb[1,0]))
ymax = max(abs(lb[0,1]), abs(lb[1,1]))
if (xmax > ymax):
    unitrect = (xmax, ss.unitcell_volume/xmax)
else:
    unitrect = (ss.unitcell_volume/ymax, ymax)

In [18]:
# Create the finite version of ss, ssw
repeatrange = ((-60,61),(-60,61)) #PARAM
latshifts = [np.arange(*repeatrange[i])[:, None] * ss.lattice_basis[i] for i in (0,1)]
latpoints = latshifts[0][:,None,:] + latshifts[1][None,:,:]
latpoints.shape = (latpoints.shape[0]*latpoints.shape[1],3)

#latpoints = np.concatenate((latpoints[:220],latpoints[221:])) # DELETHE

particles_finite = []
for lp in latpoints:
    for pi, pos in enumerate(data['ss_positions']):
        bspec = BaseSpec(data['ss_bspecs_flat'][offsets[pi]:offsets[pi]+nelems[pi]])
        particles_finite.append(Particle(lp+pos, tmg_dummy, bspec))
ss_finite, ssw_finite = ScatteringSystem.create(particles_finite, medium, default_omega)


In [19]:
eigval_i = 0 # PARAM!
omega = data['eigval'][eigval_i]
fvc = data['eigvec'][eigval_i]
len_fvc = len(fvc)

In [20]:
ssw = ss(omega)
sswk = ssw._sswk(wavevector)

In [21]:
ssw_finite = ss_finite(omega)
fvc_finite = np.zeros((len_fvc * len(latpoints),), dtype=complex)

# THIS IS NOT GENERAL, but only for gamma point single-particle case!!! TODO FIXME!
for lpi, lp in enumerate(latpoints):
    phasefac = 1 # TODO
    fvc_finite[lpi*len_fvc:(lpi+1)*len_fvc] = fvc * phasefac


In [22]:
ss_finite.positions[220]

array([-3.304e-05,  2.184e-05,  0.000e+00])

In [23]:
ucfactor = 2.2 # "unit cell coverage"; unitrect sizes will be multiplied by this# PARAM!
zfactor = 5 # z range size / sqrt(unitcell_volume) size aspect ratio
resolution = 60 # PARAM!
sqrt_ucvol = math.sqrt(ss.unitcell_volume)
gridsz = [math.ceil(resolution * unitrect[i] / sqrt_ucvol) for i in (0,1)] #x, y
gridsz.append(math.ceil(zfactor / ucfactor * resolution)) # z
gridextent = [[ -(ucfactor * unitrect[i])/2, (ucfactor * unitrect[i] / 2) ] for i in (0,1)]
gridextent.append( [-(zfactor * sqrt_ucvol) / 2, (zfactor * sqrt_ucvol) / 2] )
gridlines = [np.linspace(*gridextent[i], gridsz[i]) for i in range(3)]
gridspacings = [gridlines[i][1]-gridlines[i][0] for i in range(3)] # should be very similar

gridoffsets = [0.,0.,0.] # PARAM!

xygrid = np.stack(np.meshgrid(gridlines[0], gridlines[1], gridoffsets[2], indexing='ij')).T
xygrid.shape=xygrid.shape[1:]
xygrid_s = xygrid - np.array([*gridspacings[:2], 0])/2 # correction of pcolormesh coordinates
xzgrid = np.stack(np.meshgrid(gridlines[0], gridoffsets[1], gridlines[2], indexing='ij')).T
xzgrid.shape = (gridsz[2], gridsz[0], 3)
xzgrid_s = xzgrid - np.array([gridspacings[0], 0, gridspacings[2]])/2

#for imshow
lenscale = 1000e-9 #PARAM!
xyextent = [gridextent[i][j]/lenscale for (i,j) in [(0,0),(0,1),(1,0),(1,1)]]
xzextent = [gridextent[i][j]/lenscale for (i,j) in [(0,0),(0,1),(2,0),(2,1)]]

In [24]:
logging.info("Start calculating xy field basis")
xy_basis = np.zeros(xygrid.shape[:-1] + (len(bspec), 3), dtype=complex)
for pi in range(ss_finite.particle_count):
    xy_basis += ssw_finite.scattered_field_basis(xygrid, particle=pi)
logging.info("Start calculating xz field basis")
xz_basis = np.zeros(xzgrid.shape[:-1] + (len(bspec), 3), dtype=complex)
for pi in range(ss_finite.particle_count):
    xz_basis += ssw_finite.scattered_field_basis(xzgrid, particle=pi)

logging.info("Basis fields done")


2020-07-26 13:30:50,646 Start calculating xy field basis
2020-07-26 13:36:17,321 Start calculating xz field basis
2020-07-26 13:46:14,863 Basis fields done


In [25]:
xy_basis.size

324000

In [26]:
cutoff_ratio_default = 0.03
def cutfield(a, cutoff_ratio=cutoff_ratio_default):
    vmin = np.nanquantile(a, cutoff_ratio)
    vmax = np.nanquantile(a, 1-cutoff_ratio)
    return np.clip(a, vmin, vmax)
    

In [29]:
%matplotlib inline
lwfac = 5 #PARAM!
plotfile = "basisfields_%dx%d_directsum.pdf" % (repeatrange[0][1]-repeatrange[0][0], repeatrange[1][1]-repeatrange[1][0]) # PARAM

t,l,m = bspec.tlm()
with PdfPages(plotfile) as pdf:
  for i in range(ss.fecv_size):
    xyE = cleanarray(xy_basis[...,i,:])
    xzE = cleanarray(xz_basis[...,i,:])
    f = plt.figure(figsize=(16, 10))
    f.suptitle("(%s, %d, %d)" % ('E' if t[i] == 2 else 'M', l[i], m[i]))
    grid = AxesGrid(f, 111,  # similar to subplot(111)
                    nrows_ncols=(2, 4),
                    axes_pad=(0.45, 0.65),
                    label_mode="L",
                    share_all=False,
                    cbar_location="right",
                    cbar_mode="each",
                    cbar_size="7%",
                    cbar_pad="2%",
                    )
    # xy part
    ax = grid[0:4]
    cbax = grid.cbar_axes[0:4]

    for axi in range(4):
        ax[axi].set_xlim(xyextent[:2])
        ax[axi].set_ylim(xyextent[2:])
    flim = max(np.amax(cutfield(abs(xyE[...,2].real))), np.amax(cutfield(abs(xyE[...,2].imag)))) #this is stupid
    print(flim)
    if (flim != 0):
        axi=0
        c = ax[axi].imshow(cutfield(xyE[...,2].real), origin='lower', cmap=cm.bwr, extent=xyextent, interpolation='none', vmin = -flim, vmax=flim)#, vmin=vmin, vmax=vmax, interpolation='none')
        cb = cbax[axi].colorbar(c)
        axi=1
        c = ax[axi].imshow(cutfield(xyE[...,2].imag), origin='lower', cmap=cm.bwr, extent=xyextent, interpolation='none', vmin = -flim, vmax=flim)#, vmin=vmin, vmax=vmax, interpolation='none')
        cb = cbax[axi].colorbar(c)

    axi=2
    field = xyE.real[...,:2]
    speed = cutfield(np.linalg.norm(field, axis=-1), 0.2)
    if(np.any(speed != 0)):
        lw = 1# lwfac*speed/np.amax(speed)
        c = ax[axi].imshow(speed, origin='lower', extent=xyextent, cmap=cm.magma, vmin=0, vmax=np.nanquantile(speed, 1-cutoff_ratio_default), interpolation='none')
        ax[axi].streamplot(xygrid[...,0]/lenscale, xygrid[...,1]/lenscale, field[...,0], field[...,1],
                              linewidth=lw,color='r'                     )
        cb = cbax[axi].colorbar(c)

    axi=3
    field = xyE.imag[...,:2]
    speed = cutfield(np.linalg.norm(field, axis=-1), 0.2)
    if(np.any(speed != 0)):
        lw = 1# lwfac*speed/np.amax(speed)
        c = ax[axi].imshow(speed, origin='lower', extent=xyextent, cmap=cm.magma, vmin=0, vmax=np.nanquantile(speed, 1-cutoff_ratio_default), interpolation='none')
        ax[axi].streamplot(xygrid[...,0]/lenscale, xygrid[...,1]/lenscale, field[...,0], field[...,1],
                              linewidth=lw,color='r'                     )
        cb = cbax[axi].colorbar(c)

    # xz part
    ax = grid[4:8]
    cbax = grid.cbar_axes[4:8]
    for axi in range(4):
        ax[axi].set_xlim(xzextent[:2])
        ax[axi].set_ylim(xzextent[2:])

    axi=0
    flim = max(np.nanmax(cutfield(abs(xzE[...,1].real))), np.nanmax(cutfield(abs(xzE[...,1].imag))))  #this is stupid
    print(flim)
    if (flim != 0):
        c = ax[axi].imshow(cutfield(xzE[...,1].real), origin='lower', cmap=cm.bwr, extent=xzextent, interpolation='none', vmin = -flim, vmax=flim)#, vmin=vmin, vmax=vmax, interpolation='none')
        cb = cbax[axi].colorbar(c)
        axi=1
        c = ax[axi].imshow(cutfield(xzE[...,1].imag), origin='lower', cmap=cm.bwr, extent=xzextent, interpolation='none', vmin = -flim, vmax=flim)#, vmin=vmin, vmax=vmax, interpolation='none')
        cb = cbax[axi].colorbar(c)

    axi=2
    field = xzE.real[...,::2]
    speed = cutfield(np.linalg.norm(field, axis=-1), 0.05)
    if(np.any(speed != 0)):
        lw = 1# lwfac*speed/np.amax(speed)
        c = ax[axi].imshow(speed, origin='lower', extent=xzextent, cmap=cm.magma, vmin=0, vmax=np.nanquantile(speed, 1-cutoff_ratio_default), interpolation='none')
        ax[axi].streamplot(xzgrid[...,0]/lenscale, xzgrid[...,2]/lenscale, field[...,0], field[...,1],
                              linewidth=lw,color='r'                     )
        cb = cbax[axi].colorbar(c)

    axi=3
    field = xzE.imag[...,::2]
    speed = cutfield(np.linalg.norm(field, axis=-1), 0.05)
    if(np.any(speed != 0)):
        lw = 1# lwfac*speed/np.amax(speed)
        c = ax[axi].imshow(speed, origin='lower', extent=xzextent, cmap=cm.magma, vmin=0, vmax=np.nanquantile(speed, 1-cutoff_ratio_default), interpolation='none')
        ax[axi].streamplot(xzgrid[...,0]/lenscale, xzgrid[...,2]/lenscale, field[...,0], field[...,1],
                              linewidth=lw,color='r'                     )
        cb = cbax[axi].colorbar(c)
    pdf.savefig(f)
    plt.close(f)


0.0
0.8576424317168352
2.5631897466909934
0.0004249888747223733
0.0
0.8576424317168377
0.0
1.0779759071366666
23.771260784066907
0.4182581953807909
0.0
0.00020324625311503812
23.77126078406683
0.4182265864707979
0.0
1.077916728423413
0.0
2.454630322187389
607.9274769596968
1.9331833803570833
0.0
2.568281564854913
952.6281666806746
0.0003703452528865735
0.0
2.568281564854913
607.9274769596972
1.9331631629313715
0.0
2.454639362747998
1.2422826319310758
0.19007199183019008
0.0
1.364295876759709
1.242282631931075
0.19007199183019008
7.814910110273082
0.20071723537625655
0.0
1.2959093983932921
0.0
0.49144008040951204
0.0
1.2960740277923586
7.8155812649170935
0.20061271567875816
72.82910567368732
0.4003379026609176
0.0
1.294600426849411
18.133806945843684
1.3015665928128195
0.0
2.211750332516779
18.13438887353312
1.3015665928128195
0.0
1.2946004268494078
72.82908075868862
0.4003379026609175


In [28]:
repeatrange

((-60, 61), (-60, 61))